### AI/LLM Engineering Kick-off!! 


For our initial activity, we will be using the OpenAI Library to Programmatically Access GPT-4.1-nano!

In order to get started, you'll need an OpenAI API Key. [here](https://platform.openai.com)!

In [19]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/chat) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `developer`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-BzbKsZipLEs4pTx9dI97U6InAIB2G', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex (formerly known as GPT Index) are both popular frameworks designed to facilitate building applications that leverage large language models (LLMs), but they serve different primary purposes and have distinct features. Here\'s a high-level comparison:\n\n**1. Purpose and Focus**\n\n- **LangChain:**  \n  Primarily a framework for building, managing, and deploying LLM-powered applications with a focus on composability, prompt management, and multi-step workflows. It provides tools for chains (sequence of calls), memory management, agents (dynamic decision making), and integrations with various APIs and data sources.\n\n- **LlamaIndex (GPT Index):**  \n  Focused on enabling easy creation of indices over external data sources (like documents, PDFs, databases) so that LLMs can efficiently retrieve and reason o

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [4]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-4.1-nano") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "developer", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [5]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

Great question! LangChain and LlamaIndex (formerly known as GPT Index) are both popular frameworks designed to facilitate the development of language model applications, especially for tasks involving document ingestion, retrieval, and chatbot creation. However, they have different focuses, architectures, and use cases. Here's an overview of their main differences:

**1. Purpose and Focus**

- **LangChain:**
  - A comprehensive framework primarily aimed at building applications that leverage large language models (LLMs) for complex workflows.
  - Focuses on connecting LLMs with external tools, APIs, data sources, and managing conversational states.
  - Supports multi-step reasoning, prompt management, memory, and chaining multiple operations together.
  - Suitable for building chatbots, question-answering systems, and integrated AI applications.

- **LlamaIndex (GPT Index):**
  - Designed specifically for indexing and querying large document collections to enable efficient retrieval using LLMs.
  - Focuses on building indices over unstructured data (text, PDFs, CSVs, etc.) to perform question-answering and information retrieval.
  - Acts as a bridge between raw data sources and LLMs for context-aware responses.

**2. Architecture and Components**

- **LangChain:**
  - Provides high-level abstractions such as Chains, Agents, Prompts, Memory, and tools.
  - Modular design allows for flexible composition of workflows.
  - Supports multiple models, prompt templates, and external integrations.
  
- **LlamaIndex:**
  - Focuses on data ingestion, embedding generation, indexing, and retrieval.
  - Uses constructs like indices (e.g., GPTSimpleVectorIndex, HierarchicalIndex) to organize data.
  - Integrates with vector databases and supports retrieval-augmented generation (RAG) workflows.

**3. Use Cases**

- **LangChain:**
  - Building chatbots with multi-turn conversations.
  - Automating complex workflows involving third-party APIs.
  - Managing stateful interactions and multi-step reasoning.
  
- **LlamaIndex:**
  - Creating a document search or question-answering system.
  - Building knowledge bases from unstructured data.
  - Quick retrieval of relevant documents to provide context for LLMs.

**4. Integration with LLMs**

- **Both frameworks** facilitate interacting with LLMs like OpenAI's GPT models.
- **LangChain** emphasizes chaining and logic flow, making it ideal for applications requiring dynamic interactions and tool use.
- **LlamaIndex** emphasizes retrieval of relevant data before passing it to the LLM for answering.

---

**Summary Table:**

| Aspect                | LangChain                                           | LlamaIndex (GPT Index)                          |
|-----------------------|-----------------------------------------------------|------------------------------------------------|
| Primary Focus         | Application workflows, chaining, tools, conversation | Data ingestion, indexing, retrieval for documents |
| Use Cases             | Chatbots, multi-step workflows, integrations       | Document QA, knowledge bases, info retrieval |
| Architecture          | Modular chains, agents, prompts, memory            | Data indexing, vector retrieval, embedding management |
| Data Handling         | External API integrations, multi-modal data       | Large document collections, unstructured data |
| Interaction with LLM | Workflow orchestration, prompting, reasoning       | Retrieval augmented generation, context provision |

---

**In brief:**

- **Use LangChain** if you're building complex conversational AI, applying reasoning, or orchestrating multiple tools and data sources.
- **Use LlamaIndex** if your goal is to build a system that efficiently indexes large document repositories and enables quick, accurate retrieval and question-answering over unstructured data.

Let me know if you'd like more detail on how to use either of these frameworks!

Let's focus on extending this a bit, and incorporate a `developer` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The `developer` message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the `developer` prompt.

In [6]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

Are you kidding me? I don't have time to waste on petty ice preferences when I'm starving and furious about it! Just give me whatever is available—crushed, cubed, it doesn't matter—just get me something to eat already!

Let's try that same prompt again, but modify only our system prompt!

In [7]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I think crushed ice has a fun, refreshing crunch that’s perfect for drinks like cocktails or snow cones, while cubed ice keeps beverages colder longer and is great for sipping. Both have their charm—depends on the mood! Which do you prefer?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [8]:
print(joyful_response)

ChatCompletion(id='chatcmpl-BzbLSnxD9mHn5laTN4vcrTEHi6hTZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I think crushed ice has a fun, refreshing crunch that’s perfect for drinks like cocktails or snow cones, while cubed ice keeps beverages colder longer and is great for sipping. Both have their charm—depends on the mood! Which do you prefer?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754021262, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_38343a2f8f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=30, total_tokens=80, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


### Prompt Engineering

Now that we have a basic handle on the `developer` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-4.1-mini` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [9]:
list_of_prompts = [
    user_prompt("Write a brief text on climate change.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

Climate change refers to long-term shifts in temperature, precipitation, and other atmospheric patterns caused primarily by human activities such as burning fossil fuels, deforestation, and industrial processes. These activities increase greenhouse gases like carbon dioxide in the atmosphere, leading to global warming. The impacts of climate change include rising sea levels, more frequent and severe weather events, melting glaciers, and disruptions to ecosystems and agriculture. Addressing climate change requires collective actions to reduce emissions, transition to renewable energy sources, and promote sustainable practices worldwide.

In [10]:
list_of_prompts = [
    user_prompt("Write a brief text on climate change as vice ganda in a talk show.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

Ay, nako, mga kaibigan! Alam nyo, kahit saan tayo pumunta, napapansin ko, ang usapin ng climate change, e, parang trending na rin—pero hindi maganda ang meaning, ha! Kasi itong climate change, parang nagkakalat na hindi natin naiiwasan, gaya ng uso—pero ito, hindi uso, seryoso ito! Umiinit, tumataas ang tubig baha, tagtuyot, at sakit sa panahon, parang nakikiusap na ang mundo na mag-ingat tayo. Kailangan talaga natin maging responsible, mag-recycle, mag-ipon ng lakas para sa kinabukasan ng planeta. Dahil kung hindi, baka bukas, wala na tayong mapuntahan kundi ang... climate apocalypse! Kaya mga kababayan, sama-sama tayo, magkaisa at maging love and care sa Earth natin. Lagi nating tandaan, ang pagbabago ay nagsisimula sa atin—kaya simulan na natin ngayon!

### ❓ Activity #1: Play around with the prompt using any techniques from the prompt engineering guide.

### Few-shot Prompting

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [11]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

The stimple wrench easily turned the falbean bolt, securing the parts tightly.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought

You'll notice that, by default, the model uses Chain of Thought to answer difficult questions!

> This pattern is leveraged even more by advanced reasoning models like [`o3` and `o4-mini`](https://openai.com/index/introducing-o3-and-o4-mini/)!

In [12]:

reasoning_problem = """
how many r's in "strawberry?" {instruction}
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

There are 2 r's in "strawberry."

Notice that the model cannot count properly. It counted only 2 r's.

### ❓ Activity #2: Update the prompt so that it can count correctly.

In [18]:

reasoning_problem = """
how many r's in "strawberry?" {instruction}
"""

list_of_prompts = [
    user_prompt(reasoning_problem),
    assistant_prompt("In error, we split it into e-r-r-o-r and we can count that the string has 3 r's"),
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

There are 3 "r"s in "strawberry."

### Conclusion

Now that you're accessing `gpt-4.1-nano` through an API, developer style, let's move on to creating a simple application powered by `gpt-4.1-nano`!

Materials adapted for PSI AI Academy. Original materials from AI Makerspace.